# 시작하기 

### (1) DB불러오기 

In [1]:
from jbfg.pandas_utils import *

import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'  
plt.rcParams['axes.unicode_minus'] = False

In [2]:
schema = 'l2_jbbk_extnl'

In [3]:
sql_1 = 'SELECT * FROM products_info'
sql_2 = 'SELECT * FROM train_x_past'
sql_3 = 'SELECT * FROM train_x_current'
sql_4 = 'SELECT * FROM train_y'

In [4]:
df_products_info   = get_df(sql_1,schema)
df_train_x_past    = get_df(sql_2,schema)
df_train_x_current = get_df(sql_3,schema)
df_train_y         = get_df(sql_4,schema)

In [38]:
# 상품 정보 테이블
df_products_info.tail()

,product_id,name,category_id,division_id,category_name,division_name
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce
99,24852,Banana,24,4,fresh fruits,produce


In [39]:
# training data - 과거 구매 내역
df_train_x_past.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
1253967,1902783,163819,47,0,19,13.0,2,1,24852
1253968,1969590,163819,42,1,15,6.0,3,1,24852
1253969,2549215,163819,9,1,11,7.0,1,1,24852
1253970,3103211,163819,39,1,14,7.0,3,1,24852
1253971,3207262,163819,8,1,12,7.0,1,1,24852


In [40]:
# training data - 최근 구매 내역
df_train_x_current.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
24419,3184000,174046,100,5,7,2.0,12,1
24420,1754722,6710,100,5,11,3.0,21,1
24421,1076973,150840,100,0,13,0.0,18,1
24422,1247049,99707,100,1,13,3.0,6,1
24423,1775787,201038,100,5,9,2.0,1,1


In [41]:
# training data - 최근 주문 상품
df_train_y.tail()

,product_id
24419,24852
24420,24852
24421,24852
24422,24852
24423,24852


In [42]:
# 과거 구매내역 테이블에 해당 상품 정보 Join
df_train_x_past_pd_info = pd.merge(df_train_x_past, df_products_info, on = ['product_id'], how = 'outer')
df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs


In [43]:
# 최근 구매내역 테이블에 해당 상품 정보 Join
df_train_x_current_pd_info = df_train_x_current.join(df_train_y) # df_train_x_current data의 각 row에 매핑되는 상품정보로 단순 Join 실시!
df_train_x_current_pd_info = pd.merge(df_train_x_current_pd_info, df_products_info, on = ['product_id'], how = 'outer')
df_train_x_current_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name
24419,3184000,174046,100,5,7,2.0,12,1,24852,Banana,24,4,fresh fruits,produce
24420,1754722,6710,100,5,11,3.0,21,1,24852,Banana,24,4,fresh fruits,produce
24421,1076973,150840,100,0,13,0.0,18,1,24852,Banana,24,4,fresh fruits,produce
24422,1247049,99707,100,1,13,3.0,6,1,24852,Banana,24,4,fresh fruits,produce
24423,1775787,201038,100,5,9,2.0,1,1,24852,Banana,24,4,fresh fruits,produce


#### 최근 구매내역 정보를 바탕으로 고객별 상품주문 이력 여부 Data Frame 재생성 → 고객번호별(distinct) 상품주문 이력

In [44]:
# 1) 필요한 데이터 칼럼들만 가져오기
df_train_x_current_new_data = df_train_x_current_pd_info[['cstno','division_name']]
df_train_x_current_new_data

,cstno,division_name
0,41116,beverages
1,127113,beverages
2,140168,beverages
3,60604,beverages
4,136554,beverages
...,...,...
24419,174046,produce
24420,6710,produce
24421,150840,produce
24422,99707,produce


In [45]:
# 2) 고객번호별 최근 구매내역 상품 행렬 변환 → 과거 구매내역 테이블에 고객번호(Distinct) Join 하기 위함 → 신규 칼럼 생성
# 2-1) 상품 칼럼의 유효값 조회
df_train_x_current_new_data.division_name.unique()

array(['beverages', 'frozen', 'dairy eggs', 'produce', 'canned goods',
       'deli', 'pantry', 'meat seafood', 'bakery'], dtype=object)

#### DIVISION_NAME을 대분류로 가정하고 분류 

In [46]:
# 2) 고객번호별 최근 구매내역 상품 행렬 변환 → 과거 구매내역 테이블에 고객번호(Distinct) Join 하기 위함 → 신규 칼럼 생성
# 2-2) 상품 칼럼별 신규 칼럼 생성
gds_nm_1  = list()
gds_nm_2  = list()
gds_nm_3  = list()
gds_nm_4  = list()
gds_nm_5  = list()
# gds_nm_6  = list()
gds_nm_7  = list()
gds_nm_8  = list()
gds_nm_9  = list()
gds_nm_10 = list()

for idx in df_train_x_current_new_data['division_name']:
    if idx == 'beverages':
        gds_nm_1.append(1)
    else:
        gds_nm_1.append(0)   
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'frozen':
        gds_nm_2.append(1)
    else:
        gds_nm_2.append(0)    
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'dairy eggs':
        gds_nm_3.append(1)
    else:
        gds_nm_3.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'produce':
        gds_nm_4.append(1)
    else:
        gds_nm_4.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'canned goods':
        gds_nm_5.append(1)
    else:
        gds_nm_5.append(0)     
# for idx in df_train_x_current_new_data['division_name']:
#     if idx == 'packaged vegetables fruits':
#         gds_nm_6.append(1)
#     else:
#         gds_nm_6.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'deli':
        gds_nm_7.append(1)
    else:
        gds_nm_7.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'pantry':
        gds_nm_8.append(1)
    else:
        gds_nm_8.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'meat seafood':
        gds_nm_9.append(1)
    else:
        gds_nm_9.append(0)     
for idx in df_train_x_current_new_data['division_name']:
    if idx == 'bakery':
        gds_nm_10.append(1)
    else:
        gds_nm_10.append(0)             
        
df_train_x_current_new_data['beverages'] = gds_nm_1
df_train_x_current_new_data['frozen'] = gds_nm_2
df_train_x_current_new_data['dairy eggs'] = gds_nm_3
df_train_x_current_new_data['produce'] = gds_nm_4
df_train_x_current_new_data['canned goods'] = gds_nm_5
# df_train_x_current_new_data['packaged vegetables fruits'] = gds_nm_6
df_train_x_current_new_data['deli'] = gds_nm_7
df_train_x_current_new_data['pantry'] = gds_nm_8
df_train_x_current_new_data['meat seafood'] = gds_nm_9
df_train_x_current_new_data['bakery'] = gds_nm_10
df_train_x_current_new_data

/opt/conda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,cstno,division_name,beverages,frozen,dairy eggs,produce,canned goods,deli,pantry,meat seafood,bakery
0,41116,beverages,1,0,0,0,0,0,0,0,0
1,127113,beverages,1,0,0,0,0,0,0,0,0
2,140168,beverages,1,0,0,0,0,0,0,0,0
3,60604,beverages,1,0,0,0,0,0,0,0,0
4,136554,beverages,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
24419,174046,produce,0,0,0,1,0,0,0,0,0
24420,6710,produce,0,0,0,1,0,0,0,0,0
24421,150840,produce,0,0,0,1,0,0,0,0,0
24422,99707,produce,0,0,0,1,0,0,0,0,0


In [47]:
# 2) 고객번호별 최근 구매내역 상품 행렬 변환 → 과거 구매내역 테이블에 고객번호(Distinct) Join 하기 위함 → 신규 칼럼 생성
# 2-3) 고객별 중복 제거
del df_train_x_current_new_data['division_name'] # 중복 제거 전 기존 division_name 칼럼 삭제
df_train_x_current_new_data = df_train_x_current_new_data.drop_duplicates(keep = False, ignore_index = True)
df_train_x_current_new_data

,cstno,beverages,frozen,dairy eggs,produce,canned goods,deli,pantry,meat seafood,bakery
0,127113,1,0,0,0,0,0,0,0,0
1,60604,1,0,0,0,0,0,0,0,0
2,136554,1,0,0,0,0,0,0,0,0
3,30098,1,0,0,0,0,0,0,0,0
4,42711,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4770,84092,0,0,0,1,0,0,0,0,0
4771,74315,0,0,0,1,0,0,0,0,0
4772,181540,0,0,0,1,0,0,0,0,0
4773,172768,0,0,0,1,0,0,0,0,0


In [50]:
# 2) 고객번호별 최근 구매내역 상품 행렬 변환 → 과거 구매내역 테이블에 고객번호(Distinct) Join 하기 위함 → 신규 칼럼 생성
# 2-4) 고객별 중복 검증
df_train_x_current_new_data.duplicated(['cstno'])

0       False
1       False
2       False
3       False
4       False
        ...  
4770    False
4771    False
4772    False
4773     True
4774    False
Length: 4775, dtype: bool

In [51]:
# 사례 검증) 고객별 상품별 주문 존재 시, 별도 칼럼으로 존재
df_train_x_current_new_data[df_train_x_current_new_data['cstno'] == '172768']

,cstno,beverages,frozen,dairy eggs,produce,canned goods,deli,pantry,meat seafood,bakery
2628,172768,0,0,1,0,0,0,0,0,0
3506,172768,1,0,0,0,0,0,0,0,0
4773,172768,0,0,0,1,0,0,0,0,0


In [52]:
# cstno별 groupby sum 실시
df_train_x_current_new_data = df_train_x_current_new_data.groupby('cstno',as_index = False).sum()
df_train_x_current_new_data

,cstno,beverages,frozen,dairy eggs,produce,canned goods,deli,pantry,meat seafood,bakery
0,10000,0,0,0,1,0,0,0,0,0
1,100001,1,0,1,0,0,0,0,0,0
2,100071,1,0,0,0,0,0,0,0,0
3,100116,0,0,1,1,0,0,0,0,0
4,100142,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3606,99727,1,0,0,1,0,0,0,0,0
3607,99735,0,0,0,1,0,0,0,0,0
3608,9981,0,0,0,1,0,0,0,0,0
3609,99894,1,0,0,0,0,0,0,0,0


In [53]:
# 사례 검증 Check!
df_train_x_current_new_data[df_train_x_current_new_data['cstno'] == '172768']

,cstno,beverages,frozen,dairy eggs,produce,canned goods,deli,pantry,meat seafood,bakery
1407,172768,1,0,1,1,0,0,0,0,0


In [54]:
df_train_x_current_new_data.isnull().sum()

cstno           0
beverages       0
frozen          0
dairy eggs      0
produce         0
canned goods    0
deli            0
pantry          0
meat seafood    0
bakery          0
dtype: int64

### <font color = 'red'>분석에 필요한 신규 Data 칼럼 추가

In [55]:
# 1) (과거 구매 내역 테이블) 주문요일 변경
week_dvcd_1 = list()
week_dvcd_2 = list()

for idx in df_train_x_past_pd_info['week_day']:
    if idx == '0':
        week_dvcd_1.append('7.일요일')
    elif idx == '1':
        week_dvcd_1.append('1.월요일')        
    elif idx == '2':
        week_dvcd_1.append('2.화요일')
    elif idx == '3':
        week_dvcd_1.append('3.수요일')
    elif idx == '4':
        week_dvcd_1.append('4.목요일')
    elif idx == '5':
        week_dvcd_1.append('5.금요일')
    else:
        week_dvcd_1.append('6.토요일')        
        
for idx in df_train_x_past_pd_info['week_day']:
    if (idx == '0') | (idx == '6'):
        week_dvcd_2.append('2.공휴일')
    else:
        week_dvcd_2.append('1.영업일')

df_train_x_past_pd_info['week_day_dvcd_1'] = week_dvcd_1
df_train_x_past_pd_info['week_day_dvcd_2'] = week_dvcd_2
df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,category_id,division_id,category_name,division_name,week_day_dvcd_1,week_day_dvcd_2
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs,5.금요일,1.영업일
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs,5.금요일,1.영업일
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs,5.금요일,1.영업일
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs,6.토요일,2.공휴일
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,86,16,eggs,dairy eggs,7.일요일,2.공휴일


In [56]:
# NULL값 검증
df_train_x_past_pd_info.week_day_dvcd_1.unique()

array(['1.월요일', '6.토요일', '5.금요일', '3.수요일', '7.일요일', '4.목요일', '2.화요일'],
      dtype=object)

In [57]:
# NULL값 검증
df_train_x_past_pd_info.week_day_dvcd_2.unique()

array(['1.영업일', '2.공휴일'], dtype=object)

In [58]:
# 관련 변수 모두 숫자형으로 변경
df_train_x_past_pd_info['order_number_int']     = pd.to_numeric(df_train_x_past_pd_info['order_number'])
df_train_x_past_pd_info['week_day_int']     = pd.to_numeric(df_train_x_past_pd_info['week_day'])
df_train_x_past_pd_info['order_hour_int']     = pd.to_numeric(df_train_x_past_pd_info['order_hour'])
df_train_x_past_pd_info['order_interval_int'] = pd.to_numeric(df_train_x_past_pd_info['order_interval'])
df_train_x_past_pd_info['add_order_int']     = pd.to_numeric(df_train_x_past_pd_info['add_order'])
df_train_x_past_pd_info['is_reordered_int']     = pd.to_numeric(df_train_x_past_pd_info['is_reordered'])

df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,category_name,division_name,week_day_dvcd_1,week_day_dvcd_2,order_number_int,week_day_int,order_hour_int,order_interval_int,add_order_int,is_reordered_int
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,...,eggs,dairy eggs,5.금요일,1.영업일,30,5,8,7.0,2,1
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,...,eggs,dairy eggs,5.금요일,1.영업일,33,5,7,5.0,1,1
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,...,eggs,dairy eggs,5.금요일,1.영업일,31,5,8,19.0,2,0
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,...,eggs,dairy eggs,6.토요일,2.공휴일,31,6,13,4.0,11,0
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,...,eggs,dairy eggs,7.일요일,2.공휴일,16,0,12,5.0,1,0


In [59]:
# 2) (과거 구매 내역 테이블) 주문시간대 변경
time_dvcd = list()
# 새벽 : 00시~06시 / 오전 : 06시~11시 / 점심 : 11시~13시 / 오후 : 13시~18시 / 저녁 : 18시~24시
for idx in df_train_x_past_pd_info['order_hour_int']:
    if idx >= 21:
        time_dvcd.append('8.저녁(21~24시)')
    elif idx >= 18:
        time_dvcd.append('7.저녁(18~21시)')
    elif idx >= 15:
        time_dvcd.append('6.오후(15~18시)')
    elif idx >= 13:
        time_dvcd.append('5.오후(13~15시)')
    elif idx >= 11:
        time_dvcd.append('4.점심(11~13시)')
    elif idx >= 9:
        time_dvcd.append('3.오전(09~11시)')
    elif idx >= 6:
        time_dvcd.append('2.오전(06~09시)')
    elif idx >= 0:
        time_dvcd.append('1.새벽(00~06시)')        
    else:
        time_dvcd.append('NAN')

df_train_x_past_pd_info['order_hour_dvcd'] = time_dvcd
df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,division_name,week_day_dvcd_1,week_day_dvcd_2,order_number_int,week_day_int,order_hour_int,order_interval_int,add_order_int,is_reordered_int,order_hour_dvcd
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,...,dairy eggs,5.금요일,1.영업일,30,5,8,7.0,2,1,2.오전(06~09시)
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,...,dairy eggs,5.금요일,1.영업일,33,5,7,5.0,1,1,2.오전(06~09시)
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,...,dairy eggs,5.금요일,1.영업일,31,5,8,19.0,2,0,2.오전(06~09시)
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,...,dairy eggs,6.토요일,2.공휴일,31,6,13,4.0,11,0,5.오후(13~15시)
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,...,dairy eggs,7.일요일,2.공휴일,16,0,12,5.0,1,0,4.점심(11~13시)


In [60]:
# NULL값 검증
df_train_x_past_pd_info.order_hour_dvcd.unique()

array(['3.오전(09~11시)', '4.점심(11~13시)', '5.오후(13~15시)', '7.저녁(18~21시)',
       '2.오전(06~09시)', '6.오후(15~18시)', '8.저녁(21~24시)', '1.새벽(00~06시)'],
      dtype=object)

In [61]:
# 3) (과거 구매 내역 테이블) 재주문까지의 경과일수 요약
interval_dvcd = list()

for idx in df_train_x_past_pd_info['order_interval_int']:
    if idx <= 5:
        interval_dvcd.append('1.5일내')
    elif idx <= 10:
        interval_dvcd.append('2.10일내')
    elif idx <= 15:
        interval_dvcd.append('3.15일내')
    elif idx <= 20:
        interval_dvcd.append('4.20일내')
    elif idx <= 30:
        interval_dvcd.append('5.30일내')        
    elif idx >  30:
        interval_dvcd.append('6.30일초과')                
    else:
        interval_dvcd.append('NAN')

df_train_x_past_pd_info['order_interval_dvcd'] = interval_dvcd
df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,week_day_dvcd_1,week_day_dvcd_2,order_number_int,week_day_int,order_hour_int,order_interval_int,add_order_int,is_reordered_int,order_hour_dvcd,order_interval_dvcd
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,...,5.금요일,1.영업일,30,5,8,7.0,2,1,2.오전(06~09시),2.10일내
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,...,5.금요일,1.영업일,33,5,7,5.0,1,1,2.오전(06~09시),1.5일내
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,...,5.금요일,1.영업일,31,5,8,19.0,2,0,2.오전(06~09시),4.20일내
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,...,6.토요일,2.공휴일,31,6,13,4.0,11,0,5.오후(13~15시),1.5일내
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,...,7.일요일,2.공휴일,16,0,12,5.0,1,0,4.점심(11~13시),1.5일내


In [62]:
# NULL값 검증
df_train_x_past_pd_info.order_interval.unique()
# df_train_x_past_pd_info.order_interval_int.unique()
# df_train_x_past_pd_info.order_interval_dvcd.unique()

array(['3.0', '2.0', '1.0', '12.0', '8.0', '6.0', '7.0', '5.0', '30.0',
       '14.0', '4.0', '20.0', '28.0', '', '9.0', '13.0', '10.0', '0.0',
       '24.0', '18.0', '26.0', '21.0', '29.0', '11.0', '25.0', '15.0',
       '27.0', '19.0', '16.0', '17.0', '22.0', '23.0'], dtype=object)

In [63]:
# 4) count 집계를 위한 임시 칼럼 생성
seg = list()
# 새벽 : 00시~06시 / 오전 : 06시~11시 / 점심 : 11시~13시 / 오후 : 13시~18시 / 저녁 : 18시~24시
for idx in df_train_x_past_pd_info['order_hour']:
    if idx >= '0':
        seg.append(1)
    else:
        seg.append(0)

df_train_x_past_pd_info['count_value'] = seg
df_train_x_past_pd_info.tail()

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id,name,...,week_day_dvcd_2,order_number_int,week_day_int,order_hour_int,order_interval_int,add_order_int,is_reordered_int,order_hour_dvcd,order_interval_dvcd,count_value
1253967,3396218,98204,30,5,8,7.0,2,1,11520,Large Alfresco Eggs,...,1.영업일,30,5,8,7.0,2,1,2.오전(06~09시),2.10일내,1
1253968,432471,163746,33,5,7,5.0,1,1,11520,Large Alfresco Eggs,...,1.영업일,33,5,7,5.0,1,1,2.오전(06~09시),1.5일내,1
1253969,1822814,163746,31,5,8,19.0,2,0,11520,Large Alfresco Eggs,...,1.영업일,31,5,8,19.0,2,0,2.오전(06~09시),4.20일내,1
1253970,3160609,196534,31,6,13,4.0,11,0,11520,Large Alfresco Eggs,...,2.공휴일,31,6,13,4.0,11,0,5.오후(13~15시),1.5일내,1
1253971,71274,163782,16,0,12,5.0,1,0,11520,Large Alfresco Eggs,...,2.공휴일,16,0,12,5.0,1,0,4.점심(11~13시),1.5일내,1


# 현황 보고 1) 주문 시간/요일별 주문 현황

In [64]:
# 피벗 테이블 작성
pv1 = pd.pivot_table(df_train_x_past_pd_info, values = 'count_value', index = ['order_hour_dvcd'], columns = ['week_day_dvcd_1'], aggfunc = np.sum)
pv1

week_day_dvcd_1,1.월요일,2.화요일,3.수요일,4.목요일,5.금요일,6.토요일,7.일요일
order_hour_dvcd,,,,,,,
1.새벽(00~06시),3496,2767,2512,2425,2846,3100,3745
2.오전(06~09시),29071,18514,17185,17301,20034,18579,23317
3.오전(09~11시),47274,28216,24776,24730,30635,26104,38769
4.점심(11~13시),37628,25231,22703,22512,28345,25939,37760
5.오후(13~15시),34514,24415,23290,22805,27816,27309,39614
6.오후(15~18시),40898,31497,30878,31062,36087,35253,48835
7.저녁(18~21시),19677,17556,17230,18688,17592,18817,25590
8.저녁(21~24시),10896,9548,9434,11492,10148,11927,15590
